In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import xgboost as xgb

In [2]:
df = pd.read_csv('iith_foml_2020_train.csv')
df.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24,Target Variable (Discrete)
0,1404,12,64,14,3,1,1,1,110.502,35775.2,...,NaN,NaN,15.04,104,12,2,32,1409,37677.1,1
1,909,0,235,32,1,1,1,1,-40.448,35779.4,...,2200.3,4900.005,12.03,20,1,0,13,909,25239.1,1
2,654,3,175,2,1,1,1,1,-27.445,35770.4,...,1973.3,10000.004,13.01,1,1,0,13,654,27683.5,1
3,1372,12,382,14,2,0,1,0,0.001,509.2,...,NaN,NaN,NaN,313,12,10,54,1377,39363.2,0
4,786,3,199,2,1,0,1,0,0.001,612.1,...,NaN,NaN,NaN,171,1,5,11,786,40044.4,2


In [3]:
for col in df.columns:
    print(col, np.unique(df[col]).shape[0])

Feature 1 (Discrete) 989
Feature 2 (Discrete) 40
Feature 3 (Discrete) 303
Feature 4 (Discrete) 65
Feature 5 (Discrete) 14
Feature 6 (Discrete) 5
Feature 7 (Discrete) 20
Feature 8 (Discrete) 8
Feature 9 346
Feature 10 684
Feature 11 687
Feature 12 704
Feature 13 523
Feature 14 608
Feature 15 673
Feature 16 886
Feature 17 797
Feature 18 435
Feature 19 (Discrete) 259
Feature 20 (Discrete) 52
Feature 21 (Discrete) 23
Feature 22 (Discrete) 137
Feature 23 (Discrete) 994
Feature 24 994
Target Variable (Discrete) 18


In [4]:
df.isna().sum()*100/994

Feature 1 (Discrete)           0.000000
Feature 2 (Discrete)           0.000000
Feature 3 (Discrete)           0.000000
Feature 4 (Discrete)           0.000000
Feature 5 (Discrete)           0.000000
Feature 6 (Discrete)           0.000000
Feature 7 (Discrete)           0.000000
Feature 8 (Discrete)           0.000000
Feature 9                      1.408451
Feature 10                     0.100604
Feature 11                     0.100604
Feature 12                     0.100604
Feature 13                     0.100604
Feature 14                     0.100604
Feature 15                     7.243461
Feature 16                    67.303823
Feature 17                    54.929577
Feature 18                    33.199195
Feature 19 (Discrete)          0.000000
Feature 20 (Discrete)          0.000000
Feature 21 (Discrete)          0.000000
Feature 22 (Discrete)          0.000000
Feature 23 (Discrete)          0.000000
Feature 24                     0.100604
Target Variable (Discrete)     0.000000


In [5]:
#drop Feature 16, 17, 23
#Elsewhere replace nan with median
df.drop(columns = ['Feature 16', 'Feature 17','Feature 23 (Discrete)'], inplace = True)
df.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 13,Feature 14,Feature 15,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 24,Target Variable (Discrete)
0,1404,12,64,14,3,1,1,1,110.502,35775.2,...,0.172,1436.052,5000.5,15.04,104,12,2,32,37677.1,1
1,909,0,235,32,1,1,1,1,-40.448,35779.4,...,0.032,1436.111,3720.5,12.03,20,1,0,13,25239.1,1
2,654,3,175,2,1,1,1,1,-27.445,35770.4,...,0.021,1436.103,4685.4,13.01,1,1,0,13,27683.5,1
3,1372,12,382,14,2,0,1,0,0.001,509.2,...,97.541,94.844,NaN,NaN,313,12,10,54,39363.2,0
4,786,3,199,2,1,0,1,0,0.001,612.1,...,97.981,97.823,4.1,NaN,171,1,5,11,40044.4,2


In [6]:
for i in range(21):
    df[df.columns[i]].fillna(df[df.columns[i]].median(), inplace = True)
df.isna().sum()*100/994

Feature 1 (Discrete)          0.0
Feature 2 (Discrete)          0.0
Feature 3 (Discrete)          0.0
Feature 4 (Discrete)          0.0
Feature 5 (Discrete)          0.0
Feature 6 (Discrete)          0.0
Feature 7 (Discrete)          0.0
Feature 8 (Discrete)          0.0
Feature 9                     0.0
Feature 10                    0.0
Feature 11                    0.0
Feature 12                    0.0
Feature 13                    0.0
Feature 14                    0.0
Feature 15                    0.0
Feature 18                    0.0
Feature 19 (Discrete)         0.0
Feature 20 (Discrete)         0.0
Feature 21 (Discrete)         0.0
Feature 22 (Discrete)         0.0
Feature 24                    0.0
Target Variable (Discrete)    0.0
dtype: float64

In [7]:
labels = df['Target Variable (Discrete)'].value_counts()[df['Target Variable (Discrete)'].value_counts()<5]
for i in range(len(labels.index)):
    df.drop(index = df[df['Target Variable (Discrete)']==labels.index[i]].index.values, inplace = True)
df.shape

(974, 22)

In [8]:
df['Target Variable (Discrete)'].value_counts()

1     488
0     249
2     109
6      70
5      41
8       7
14      5
7       5
Name: Target Variable (Discrete), dtype: int64

In [9]:
df_tst = pd.read_csv('iith_foml_2020_test.csv')
df_tst.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24
0,146,12,42,14,7,1,1,1,118.004,35693.5,...,4200.3,1900.1,NaN,NaN,44,12,2,42,146,37384.5
1,35,0,12,5,0,0,1,0,0.001,471.5,...,531.4,NaN,NaN,NaN,17,6,8,0,35,41465.1
2,1018,8,259,2,1,1,1,1,NaN,35774.5,...,5514.2,NaN,NaN,15.04,1,1,3,20,1018,37826.2
3,383,7,117,5,1,1,1,1,53.002,34094.1,...,3358.4,1100.2,14000.001,15.04,101,6,3,20,383,40277.3
4,1216,7,40,5,2,0,1,4,0.005,1471.3,...,225.1,NaN,NaN,3.02,276,6,7,43,1221,28419.5


In [10]:
df_tst.drop(columns = ['Feature 16', 'Feature 17', 'Feature 23 (Discrete)'], inplace = True)
df_tst.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 12,Feature 13,Feature 14,Feature 15,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 24
0,146,12,42,14,7,1,1,1,118.004,35693.5,...,0.025200,96.405,1435.935,4200.3,NaN,44,12,2,42,37384.5
1,35,0,12,5,0,0,1,0,0.001,471.5,...,0.001100,97.285,94.181,531.4,NaN,17,6,8,0,41465.1
2,1018,8,259,2,1,1,1,1,NaN,35774.5,...,0.000285,0.023,1436.092,5514.2,15.04,1,1,3,20,37826.2
3,383,7,117,5,1,1,1,1,53.002,34094.1,...,0.041900,0.531,1440.095,3358.4,15.04,101,6,3,20,40277.3
4,1216,7,40,5,2,0,1,4,0.005,1471.3,...,0.001530,82.502,115.504,225.1,3.02,276,6,7,43,28419.5


In [11]:
for i in range(21):
    df_tst[df_tst.columns[i]].fillna(df[df.columns[i]].median(), inplace = True)
df_tst.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 12,Feature 13,Feature 14,Feature 15,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 24
0,146,12,42,14,7,1,1,1,118.004,35693.5,...,0.025200,96.405,1435.935,4200.3,10.05,44,12,2,42,37384.5
1,35,0,12,5,0,0,1,0,0.001,471.5,...,0.001100,97.285,94.181,531.4,10.05,17,6,8,0,41465.1
2,1018,8,259,2,1,1,1,1,0.003,35774.5,...,0.000285,0.023,1436.092,5514.2,15.04,1,1,3,20,37826.2
3,383,7,117,5,1,1,1,1,53.002,34094.1,...,0.041900,0.531,1440.095,3358.4,15.04,101,6,3,20,40277.3
4,1216,7,40,5,2,0,1,4,0.005,1471.3,...,0.001530,82.502,115.504,225.1,3.02,276,6,7,43,28419.5


In [12]:
df_tst.isna().sum()

Feature 1 (Discrete)     0
Feature 2 (Discrete)     0
Feature 3 (Discrete)     0
Feature 4 (Discrete)     0
Feature 5 (Discrete)     0
Feature 6 (Discrete)     0
Feature 7 (Discrete)     0
Feature 8 (Discrete)     0
Feature 9                0
Feature 10               0
Feature 11               0
Feature 12               0
Feature 13               0
Feature 14               0
Feature 15               0
Feature 18               0
Feature 19 (Discrete)    0
Feature 20 (Discrete)    0
Feature 21 (Discrete)    0
Feature 22 (Discrete)    0
Feature 24               0
dtype: int64

In [13]:
df_tst.shape

(426, 21)

In [ ]:
Target Variable (Discrete)

In [14]:
df_tgt = df['Target Variable (Discrete)']
df_tgt

0      1
1      1
2      1
3      0
4      2
      ..
989    0
990    2
991    6
992    0
993    2
Name: Target Variable (Discrete), Length: 974, dtype: int64

In [21]:
df.drop(columns = 'Target Variable (Discrete)', inplace = True)
df.shape

(974, 21)

In [22]:
df_combined = df.append(df_tst)
df_combined.shape

(1400, 21)

In [23]:
df_combined.columns

Index(['Feature 1 (Discrete)', 'Feature 2 (Discrete)', 'Feature 3 (Discrete)',
       'Feature 4 (Discrete)', 'Feature 5 (Discrete)', 'Feature 6 (Discrete)',
       'Feature 7 (Discrete)', 'Feature 8 (Discrete)', 'Feature 9',
       'Feature 10', 'Feature 11', 'Feature 12', 'Feature 13', 'Feature 14',
       'Feature 15', 'Feature 18', 'Feature 19 (Discrete)',
       'Feature 20 (Discrete)', 'Feature 21 (Discrete)',
       'Feature 22 (Discrete)', 'Feature 24'],
      dtype='object')

In [24]:
to_categorical(df_combined['Feature 8 (Discrete)']).shape

(1400, 9)

In [35]:
encd = []

#First 8 discrete features
df_final = df_combined['Feature 1 (Discrete)']

for i in range(1, 8):
    if(df_combined[df_combined.columns[i]].unique().size < 25):
        df_final = pd.concat([df_final, pd.get_dummies(df_combined[df_combined.columns[i]], prefix = df_combined.columns[i])], axis = 1)
        #df_combined.drop(columns = df_combined.columns[i])
        encd.append(i)
    else:
        df_final = pd.concat([df_final, df_combined[df_combined.columns[i]]], axis = 1)
        
for i in range(8, 16):
    df_final = pd.concat([df_final, df_combined[df_combined.columns[i]]], axis = 1)
    
for i in range(16, 20):
    if(df_combined[df_combined.columns[i]].unique().size < 25):
        df_final = pd.concat([df_final, pd.get_dummies(df_combined[df_combined.columns[i]], prefix = df_combined.columns[i])], axis = 1)
        #df_combined.drop(columns = df_combined.columns[i])
        encd.append(i)
    else:
        df_final = pd.concat([df_final, df_combined[df_combined.columns[i]]], axis = 1)

df_final = pd.concat([df_final, df_combined[df_combined.columns[20]]], axis = 1)

df_final.shape

(1400, 86)

In [30]:
encd

[4, 5, 6, 7, -3]

In [31]:
df_final.columns

Index(['Feature 1 (Discrete)', 'Feature 2 (Discrete)', 'Feature 3 (Discrete)',
       'Feature 4 (Discrete)', 'Feature 5 (Discrete)_0',
       'Feature 5 (Discrete)_1', 'Feature 5 (Discrete)_2',
       'Feature 5 (Discrete)_3', 'Feature 5 (Discrete)_4',
       'Feature 5 (Discrete)_5', 'Feature 5 (Discrete)_6',
       'Feature 5 (Discrete)_7', 'Feature 5 (Discrete)_8',
       'Feature 5 (Discrete)_9', 'Feature 5 (Discrete)_10',
       'Feature 5 (Discrete)_11', 'Feature 5 (Discrete)_12',
       'Feature 5 (Discrete)_14', 'Feature 6 (Discrete)_0',
       'Feature 6 (Discrete)_1', 'Feature 6 (Discrete)_2',
       'Feature 6 (Discrete)_3', 'Feature 7 (Discrete)_0',
       'Feature 7 (Discrete)_1', 'Feature 7 (Discrete)_2',
       'Feature 7 (Discrete)_3', 'Feature 7 (Discrete)_4',
       'Feature 7 (Discrete)_5', 'Feature 7 (Discrete)_6',
       'Feature 7 (Discrete)_8', 'Feature 7 (Discrete)_9',
       'Feature 7 (Discrete)_10', 'Feature 7 (Discrete)_11',
       'Feature 7 (Discrete)_12'

In [32]:
df.shape

(974, 21)

In [36]:
X_trn = df_final.iloc[:974, :]
X_tst = df_final.iloc[974:, :]

print(X_trn.shape)
print(X_tst.shape)

(974, 86)
(426, 86)


In [38]:
Y = df_tgt.values
Y.shape

(974,)

In [40]:
x_trn, x_tst, y_trn, y_tst = train_test_split(X_trn, Y, test_size = 0.2, stratify = Y, random_state = 28)

print(x_trn.shape)
print(x_tst.shape)
print(y_trn.shape)
print(y_tst.shape)

(779, 86)
(195, 86)
(779,)
(195,)


In [183]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob', n_estimators = 25, max_depth=7,
                              subsample = 0.7)

In [184]:
xgb_model.fit(x_trn, y_trn)
y_pred = xgb_model.predict(x_tst)

mean_squared_error(y_tst, y_pred)

/home/ashutosh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:32:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6205128205128205

In [185]:
accuracy_score(y_tst, y_pred)

0.9282051282051282

In [186]:
accuracy_score(y_trn, xgb_model.predict(x_trn))

1.0

In [187]:
submit = xgb_model.predict(X_tst).astype('int')
submit.shape

(426,)

In [188]:
np.unique(submit, return_counts=True)

(array([ 0,  1,  2,  5,  6,  7,  8, 14]),
 array([108, 222,  48,  17,  27,   1,   1,   2]))

In [189]:
df_tst['Category'] = submit
df_tst['Id'] = df_tst.index + 1

In [190]:
df_tst[['Id', 'Category']].to_csv("XGBClassifier1.csv", index = False)